In [1]:
from pathlib import Path

import sys

sys.path.insert(0, "..")

import plot_data
import json

def create_info(task):
    cache_dir = "../cache"
    input_filename = "../files/results" + "_" + task + ".json"
    references, experiments = plot_data.MultiProvider().points(task, cache_dir, input_filename)
    #print(references["local_movement_pruning"][0])
    return references, experiments
    
    
create_info("squadv1")

({'local_movement_pruning': [{'cols': 1,
    'epochs': 10,
    'exact': 78.647114474929,
    'f1': 87.2352379495134,
    'fill_rate': 0.9,
    'inner_sparsity': 1,
    'name': 'topK_1.0_*_1_2_null_0._3e-5_1e-2_topK_constant_0._10_epochs',
    'rows': 1,
    'size': 1,
    'speedup': 1.0},
   {'cols': 1,
    'epochs': 10,
    'exact': 76.1116367076632,
    'f1': 85.0200791447152,
    'fill_rate': 0.7,
    'inner_sparsity': 1,
    'name': 'topK_1.0_*_1_2_null_0._3e-5_1e-2_topK_constant_0._10_epochs',
    'rows': 1,
    'size': 1,
    'speedup': 1.0},
   {'cols': 1,
    'epochs': 10,
    'exact': 73.2355723746452,
    'f1': 83.1623614706555,
    'fill_rate': 0.501,
    'inner_sparsity': 1,
    'name': 'topK_1.0_*_1_2_null_0._3e-5_1e-2_topK_constant_0._10_epochs',
    'rows': 1,
    'size': 1,
    'speedup': 1.0},
   {'cols': 1,
    'epochs': 10,
    'exact': 72.4597918637653,
    'f1': 82.3798006055091,
    'fill_rate': 0.301,
    'inner_sparsity': 1,
    'name': 'topK_1.0_*_1_2_null_0._3

In [2]:
abbrev_ = {"matched":"M",
           "mismatched":"MM",
           "accuracy":"Acc",
           "exact_match":"EM",
           "exact":"EM",
           "f1":"F1",
           "em":"EM",
           "Hybrid Filled large teacher": "Hybrid Filled LT"
          }

def abbrev(k):
    if k.lower() == "squadv1":
        return "SQuAD v1" 
    if k.lower() == "squadv2":
        return "SQuAD v2" 
    return abbrev_.get(k, k)



class TableWriter():
    HLINE = "\\midrule"
    def __init__(self, prefix = None, suffix = None, label = "todo", caption="todo"):
        self.lines = []
        self.prefix = prefix
        self.suffix = suffix
        self.label = label
        self.caption = caption

    def start_line(self):
        self.lines.append([])

    def add_element(self, element):
        self.lines[-1].append(str(element))        
        
    def insert_hline(self):
        self.lines.append([self.HLINE])

    def add_elements(self, elements, method = "separated"):
        if method == "separated":
            for e in elements:
                self.add_element(e)
        elif method == "slash":
            return self.add_element("/".join(map(lambda x : str(x), elements)))
        else:
            raise Exception("Unknown method")
        return
        if len(elements) == 1:
            s = elements[0]
        else:
            prefix = "\\begin{tabular}{%s}\n" % ("c" * len(elements))
            suffix = "\\end{tabular}\n"

            sub = TableWriter(prefix=prefix, suffix=suffix)
            sub.start_line()
            for e in elements:
                sub.add_element(e)
            s = sub.get_string()

        self.add_element(s)            

    def get_string(self):
        if self.prefix is None:
            columns = "".join(["l"] * len(self.lines[0]))
            columns = "@{}%s@{}" % columns
            s = "\\begin{table}[t]\n"
            s += "\\small\n"
            s += "\\begin{tabular}{ %s }\n" % columns
            s += "\\toprule\n"
        else:
            s = self.prefix

        for l in self.lines:
            if len(l) == 1:
                s += l[0]
            else:
                new_line = " & ".join(l) + "\\\\"
                s += new_line
            s += "\n"
        if self.suffix is None:
            s += "\\bottomrule\n"
            s += "\\end{tabular}\n"
            s += "\\caption{%s}\n" % self.caption
            s += "\\label{table:%s}\n" % self.label
            s += "\\end{table}\n"
        else:
            s += self.suffix

        return s


In [8]:
#Params #FLOPS Latency

def load_graph_json(task, force_filename = None, key="fill_rate"):
    root_path = Path("..") / "graphs" / task.lower() / "paper_summary"  / "logs"
    
    if force_filename is not None:
        filenames = [force_filename]
    else:
        filenames = ["Hybrid_Filled__large_teacher", "Hybrid__large_teacher", "Hybrid_Filled", "Hybrid"]
    
    for f in filenames:
        full_name = "paper_summary_%s_" % key + f + ".jsonl"
        full_name = root_path / full_name
        if full_name.exists():
            with full_name.open() as file:
                j = []
                for l in file:
                    j.append(json.loads(l))
                return f.replace("__", " ").replace("_"," "), j
    assert(False)

def table_for_task(task_human, columns_2, add_index=True):
    single_task = task_human.lower().replace(" ","")
    
    columns_1 = ["Model", "Size", "OPs$^{-1}$", "Speed"]
    if add_index:
        columns_1 = [""] + columns_1

    bert_linear_layers = 12  * 12 * 768 * (768 + 1) # Includes the bias
    distilbert_linear_layers = bert_linear_layers / 2
    tinybert_linear_layers = bert_linear_layers / 2
    mobilebert_linear_layers = bert_linear_layers / 4
    models_single = {"BERT":{"name":"bert",
                             "params":bert_linear_layers},
                     "DistilBERT":{"name":"distilbert",
                                   "params":distilbert_linear_layers},

                     "TinyBERT":{"name":"tinybert",
                                 "params":tinybert_linear_layers},
                    }
    if "squad" in single_task.lower():
        models_single["MobileBERT"] = {"name":"mobile_bert_measured",
                                       "params":mobilebert_linear_layers}

    models_continuous = ["MovementPruning"]


    tw = TableWriter(label=single_task, caption=task_human)

    tw.start_line()

    for i, param_name in enumerate(columns_1):
        tw.add_element(abbrev(param_name))

    for i, task in enumerate(columns_2):
        e = abbrev(task)    
        for task_part in columns_2[task]:
            p = abbrev(task_part)            
            tw.add_element(p)

    tw.insert_hline()
        
    if False:
        tw.start_line()

        for i, param_name in enumerate(columns_1):
            tw.add_element(" ")

        for i, task in enumerate(columns_2):
            elements = []
            for task_part in columns_2[task]:
                e = abbrev(task_part)
                elements.append(e)
            tw.add_elements(elements)

            
    def speedup_string(speedup):
        return "%0.2f" % speedup
    
    # Reference models
    for model, info in models_single.items():
        name = info["name"]
        tw.start_line()
        if add_index:
            tw.add_element("")
            
        tw.add_element(model)
        params = str(int(info["params"] / 1E6)) + "M"
        tw.add_element(params)
        theoretical_speedup = bert_linear_layers / info["params"]
        tw.add_element(speedup_string(theoretical_speedup))

        for i, task in enumerate(columns_2):
            references, experiments = create_info(task.lower())

            assert(name in references)
            d = references[name][0]

            if i == 0:
                tw.add_element(speedup_string(d["speedup"]))

            elements = []
            for task_part in columns_2[task]:
                e = d.get(task_part, "-")
                if e != "-":
                    e = "%0.2f" % e
                elements.append(e)
                tw.add_element(e)
            


    if "squad" in single_task.lower():
        if single_task == "squadv1":
            filenames = ["Hybrid_Filled", "Hybrid_Filled__large_teacher"]
        elif single_task == "squadv2":
            filenames = ["Hybrid_Filled"]
            
        jsons = []
        index_sets = []
        names = []
        for f in filenames:
            name, j = load_graph_json(single_task, force_filename=f)
            names.append(name)
            jsons.append(j)
            index_sets.append(list(range(len(j))))
            
        if single_task == "squadv1":
            index_sets = [[0, 3, -1], [0, -2, -1]]
        elif single_task == "squadv2":
            pass
            #index_sets = [[0, 3, -1], [0, -2, -1]]
        
    else:
        name, j = load_graph_json(single_task)
        names = [name]
        index_sets = [list(range(len(j)))]
        if single_task == "qqp":
            index_sets = [[0, 2, -2]]
        elif single_task == "mnli":
            index_sets = [[0, -2]]
        elif single_task == "sst2":        
            index_sets = [[3, 4, 5]]
        jsons = [j]
        
    model_index = 0
    # Own models
    for json_index, j in enumerate(jsons):
        indexes = index_sets[json_index]

        j = [j[i] for i in indexes]

        tw.insert_hline()
        for info in j:
            tw.start_line()
            if add_index:
                idx = "(%s)" % chr(ord('a') + model_index)
                model_index += 1
                tw.add_element(idx)

            tw.add_element(abbrev(names[json_index]))
            params = bert_linear_layers * info["fill_rate"]
            params = str(int(params / 1E6)) + "M"
            tw.add_element(params)

            theoretical_speedup = 1.0 / info["fill_rate"]
            tw.add_element(speedup_string(theoretical_speedup))

            speedup = ("%0.2f" % info["meta"].get("speedup")) if info["meta"].get("speedup", "1.0") != 1.0 else "-"

            tw.add_element(speedup)


            elements = []
            for task_part in columns_2[task]:
                #print(info, task_part)
                #for k,v in info["meta"].items():
                #    print(k,v)
                if task_part in ["exact_match","exact"]:
                    if single_task == "squadv1":
                        task_part = "exact_match"
                    e = info["meta"]["checkpoint"]["eval_metrics"].get(task_part, "-")
                else:
                    e = info["meta"].get(task_part, "-")
                e = "%0.2f" % e
                elements.append(e)
                tw.add_element(e)  
                

    return tw.get_string()

columns_all = {"MNLI":["matched", "mismatched"], "QQP":["f1", "accuracy"], "SST2":["accuracy"],
              "SQuAD v1":["f1", "exact"],
              "SQuAD v2":["f1", "exact"]}


for k,v in columns_all.items():
    #if k != "SQuAd v1":
    #    continue
    #if k != "MNLI":
    #    continue
    columns_2 = {k.replace(" ", ""):v}
    s = table_for_task(k, columns_2)
    print(s)
    
    
    

\begin{table}[t]
\small
\begin{tabular}{ @{}lllllll@{} }
\toprule
 & Model & Size & OPs$^{-1}$ & Speed & M & MM\\
\midrule
 & BERT & 85M & 1.00 & 1.00 & 84.60 & 83.40\\
 & DistilBERT & 42M & 2.00 & 2.06 & 82.20 & -\\
 & TinyBERT & 42M & 2.00 & 1.88 & 84.60 & 83.20\\
\midrule
(a) & Hybrid Filled & 29M & 2.92 & 2.12 & 83.71 & 84.08\\
(b) & Hybrid Filled & 15M & 5.53 & 3.11 & 82.69 & 82.72\\
\bottomrule
\end{tabular}
\caption{MNLI}
\label{table:mnli}
\end{table}

\begin{table}[t]
\small
\begin{tabular}{ @{}lllllll@{} }
\toprule
 & Model & Size & OPs$^{-1}$ & Speed & F1 & Acc\\
\midrule
 & BERT & 85M & 1.00 & 1.00 & 88.12 & 91.15\\
 & DistilBERT & 42M & 2.00 & 2.06 & - & 88.50\\
 & TinyBERT & 42M & 2.00 & 1.88 & 88.00 & 91.10\\
\midrule
(a) & Hybrid & 42M & 2.02 & - & 88.31 & 91.31\\
(b) & Hybrid & 15M & 5.49 & - & 87.42 & 90.62\\
(c) & Hybrid & 7M & 11.99 & - & 86.38 & 89.85\\
\bottomrule
\end{tabular}
\caption{QQP}
\label{table:qqp}
\end{table}

\begin{table}[t]
\small
\begin{tabular}{ @

In [4]:
tw = TableWriter(label="pruning_methods", caption="Pruning Methods")
tw.start_line()
tw.add_elements(["Method", "Attention", "FFN", "Teacher"])
tw.insert_hline()
tw.start_line()
tw.add_elements(["Block", "Block", "Block", "Base"])
tw.start_line()
tw.add_elements(["Struct", "Heads", "Dim", "Base"])
tw.start_line()
tw.add_elements(["Hybrid", "Block", "Dim", "Base"])
tw.start_line()
tw.add_elements(["Hybrid Filled", "Heads", "Dim", "Base"])
tw.start_line()
tw.add_elements(["Hybrid Filled LT", "Heads", "Dim", "Large"])
print(tw.get_string())

\begin{table}[t]
\small
\begin{tabular}{ @{}llll@{} }
\toprule
Method & Attention & FFN & Teacher\\
\midrule
Block & Block & Block & Base\\
Struct & Heads & Dim & Base\\
Hybrid & Block & Dim & Base\\
Hybrid Filled & Heads & Dim & Base\\
Hybrid Filled LT & Heads & Dim & Large\\
\bottomrule
\end{tabular}
\caption{Pruning Methods}
\label{table:pruning_methods}
\end{table}



In [7]:
def compare_values(tinybert_values, model_values):
    tinybert_values_final = []
    model_values_final = []

    def wrap(a):
        if not isinstance(a, (list, tuple)):
            a = (a,)
        return a
    def bold(t):
        return "\\textbf{%s}" % t

    for i, a in enumerate(tinybert_values):
        a = wrap(a)
        tiny = []
        model = []
        for j, tiny_value in enumerate(a):
            model_value = wrap(model_values[i])[j]
            if model_value == "-" or tiny_value > model_value:
                tiny_value = bold(tiny_value)
            else:
                model_value = bold(model_value)
            tiny.append(str(tiny_value))
            model.append(str(model_value))
        tinybert_values_final.append("/".join(tiny))
        model_values_final.append("/".join(model))
    return tinybert_values_final, model_values_final




def create_comparison_table(label, caption, tinybert_values, model_values):
    tinybert_values_final, model_values_final = compare_values(tinybert_values, model_values)        
    tw = TableWriter(label=label, caption=caption)
    tw.start_line()
    tw.add_elements(["Model", "SQuAD v1", "MNLI", "QQP", "SST2"])
    tw.start_line()
    tw.add_elements(["", "F1/EM", "M/MM", "F1", "Acc"])
    tw.insert_hline()
    tw.start_line()

    tw.add_elements(["TinyBERT"] + tinybert_values_final)
    tw.start_line()
    tw.add_elements(["Hybrid"] + model_values_final)
    return tw.get_string()

label = "tasks_hybrid_tinybert"
caption = "Hybrid pruning/TinyBERT task performance comparison."

tinybert_values = [(87.5, 79.7), (84.6, 83.2), 88.0, 93.0]
#model_values_speedup_x2 = [(88.1,80.6), (82.3, 82.7), 88.3, 92.0]
model_values_speedup_x188 = [(88.1,80.6), (83.2, 83.6), 87.8, 91.4]

s = create_comparison_table(label, caption, tinybert_values, model_values_speedup_x188)
print(s)

\begin{table}[t]
\small
\begin{tabular}{ @{}lllll@{} }
\toprule
Model & SQuAD v1 & MNLI & QQP & SST2\\
 & F1/EM & M/MM & F1 & Acc\\
\midrule
TinyBERT & 87.5/79.7 & \textbf{84.6}/83.2 & \textbf{88.0} & \textbf{93.0}\\
Hybrid & \textbf{88.1}/\textbf{80.6} & 83.2/\textbf{83.6} & 87.8 & 91.4\\
\bottomrule
\end{tabular}
\caption{Hybrid pruning/TinyBERT task performance comparison.}
\label{table:tasks_hybrid_tinybert}
\end{table}



In [6]:
label = "tasks_hybrid_filled_tinybert"
caption = "Hybrid Filled pruning/TinyBERT task performance comparison."

model_values = [(88.3,80.9), (83.0, 83.6), "-", "-"]

s = create_comparison_table(label, caption, tinybert_values, model_values)
print(s)

\begin{table}[t]
\small
\begin{tabular}{ @{}lllll@{} }
\toprule
Model & SQuAD v1 & MNLI & QQP & SST2\\
 & F1/EM & M/MM & F1 & Acc\\
\midrule
TinyBERT & 87.5/79.7 & \textbf{84.6}/83.2 & \textbf{88.0} & \textbf{93.0}\\
Hybrid & \textbf{88.3}/\textbf{80.9} & 83.0/\textbf{83.6} & - & -\\
\bottomrule
\end{tabular}
\caption{Hybrid Filled pruning/TinyBERT task performance comparison.}
\label{table:tasks_hybrid_filled_tinybert}
\end{table}



In [243]:
infos = []

def create_filled_comparaison():
    _, j_h = load_graph_json("squadv1", force_filename = "Hybrid", key="fill_rate")
    _, j_hf = load_graph_json("squadv1", force_filename = "Hybrid_Filled", key="fill_rate")
    from collections import defaultdict

    fill_rates = defaultdict(list)

    def register(fill_rates, j_, kind):
        j_.sort(key=lambda x:x["meta"]["speedup"])
        for j in j_:
            fr = int((j["meta"]["speedup"] - 1.9) * 30)
            if fr == 23:
                fr = 24
            #print(kind, fr, j["f1"], j["meta"]["speedup"])
            fill_rates[fr].append({kind:j})

    register(fill_rates, j_h, "h")
    register(fill_rates, j_hf, "hf")
    
    speedups = []
    f1s = defaultdict(list)
    last_key = -100
    
    for k, a in fill_rates.items():
        if abs(last_key - k) <= 1:
            last_key = k
            continue
        last_key = k
        
        if len(a) >= 2:
            a = [a[0], a[-1]]
            kinds = [list(e.keys())[0] for e in a]
            #print(kinds)
            #continue
            if(kinds != ["h", "hf"]):
                continue
            speedup_list = [list(e.values())[0]["meta"]["speedup"] for e in a]
            f1_list = [list(e.values())[0]["meta"]["f1"] for e in a]
            if False:
                print(k) 
                print(kinds)
                print(speedup_list)
                print(f1_list)
                print()
            
            print(speedup_list[0] - speedup_list[1])
            speedups.append(speedup_list[0])
            
            for i, kind in enumerate(kinds):
                f1s[kind].append(f1_list[i])

    if False:
        print(speedups)
        print(f1s)
    
    def format_list(l, rnd):
        rnd = "%0." + str(rnd) + "f"
        return list(map(lambda x : rnd % x, l))
    
    label="filled_gain"
    caption="Hybrid Filled gain over non filled network on SQuAD v1. Networks have the same number of parameters and so the same speedup."
    tw = TableWriter(label=label, caption=caption)
    tw.start_line()
    tw.add_elements(["Speedup"] + format_list(speedups, 1))
    tw.start_line()
    tw.add_elements(["Hybrid"] + format_list(f1s["h"], 1))
    tw.start_line()
    tw.add_elements(["Hybrid Filled"] + format_list(f1s["hf"], 1))
    tw.start_line()
    
    gain = []    
    for i, f in enumerate(f1s["h"]):
        gain.append(f1s["hf"][i] - f)
    
    tw.add_elements(["Gain"] + format_list(gain, 1))

    return tw.get_string()
        
s = create_filled_comparaison()
print(s)
        
    

-0.013843506980520726
0.008237765674629305
0.0003962466228548678
0.01616412497098496
\begin{table}[t]
\small
\begin{tabular}{ @{}lllll@{} }
\toprule
Speedup & 2.0 & 2.2 & 2.6 & 2.7\\
Hybrid & 88.1 & 87.4 & 85.3 & 86.1\\
Hybrid Filled & 88.3 & 88.1 & 87.2 & 86.8\\
Gain & 0.2 & 0.7 & 2.0 & 0.6\\
\bottomrule
\end{tabular}
\caption{Hybrid Filled gain over non filled network on SQuAD v1. Networks have the same number of parameters and so the same speedup.}
\label{table:filled_gain}
\end{table}



In [206]:
abs(-1)

1